In [1]:
import sys
import os
sys.path.append("..")
from globals import ROOT_DIR
from data_providers import TextDataProvider
import configparser
import argparse
import configparser
from torch import optim
from experiment_builder import ExperimentBuilder
from data_providers import *
import os
from models.cnn import *
from models.multilayer_perceptron import multi_layer_perceptron
config = configparser.ConfigParser()
config.read('../config.ini')
path_data = os.path.join(ROOT_DIR, config['DEFAULT']['PATH_DATA'])
path_labels = os.path.join(ROOT_DIR, config['DEFAULT']['PATH_LABELS'])

In [2]:
VERBOSE = True
def extract_data(embedding_key, embedding_level_key, seed):
    path_data = os.path.join(ROOT_DIR, config['DEFAULT']['PATH_DATA'])
    path_labels = os.path.join(ROOT_DIR, config['DEFAULT']['PATH_LABELS'])
    data_provider = TextDataProvider(path_data, path_labels)
    if embedding_level_key == 'word':
        output = data_provider.generate_word_level_embeddings(embedding_key, seed)
    elif embedding_level_key == 'char':
        output = data_provider.generate_char_level_embeddings(seed)
    else:
        output = data_provider.generate_tdidf_embeddings(seed)

    if VERBOSE:
        print("[Sizes] Training set: {}, Validation set: {}, Test set: {}".format(len(output['x_train']),
                                                                                  len(output['x_valid']),
                                                                                  len(output['x_test'])))
    return output

In [3]:
data = extract_data(embedding_key='twitter', embedding_level_key='word', seed=28)

=== Extracting annotations ===
=== Extracting tweets from JSON ===
[Stats] Removed 3/58358 labels
[Stats] Average tweet length is 17 words
[Stats] Average tweet length is 121 characters
[Stats] Average favorite count is 15
[Stats] Average retweet count is 146
[Stats] Average follower count is 710
[Sizes] Training set: 64.00%, Validation set: 16.00%, Test set: 20.00%
[Model] Using twitter embeddings
[Sizes] Training set: 64.00%, Validation set: 16.00%, Test set: 20.00%
[Sizes] Training set: 37348, Validation set: 9338, Test set: 11672


In [4]:
data['x_train'][0]

[array([-0.11015709, -0.09651561, -0.66805392,  0.23579759, -0.45216305,
         0.28787148,  0.25521424,  1.54814133, -0.21573402, -0.2412149 ,
         0.11163139,  0.45684257, -0.04500172, -0.2064522 ,  0.40169965,
        -0.54659333, -0.26741178,  0.23249655, -0.78202477, -0.33127879,
        -0.10315225, -1.21037624,  0.69031126,  0.35270697,  0.83935655,
         0.0882714 ,  0.25676577,  0.50293696, -0.09583937, -0.46496718,
         0.80902793,  0.73696452,  0.7715764 , -0.4591314 ,  0.49993722,
        -0.22930529,  0.57005216, -0.37013891, -0.57022919,  0.23241023,
         0.3707914 ,  0.50404526, -0.63205747, -0.10873196, -0.1198273 ,
         0.81948153,  0.40262368, -0.05279416, -0.07025077, -0.29875996,
         0.50665903, -0.10520409,  0.29157869,  0.32456724,  0.75495838,
        -0.14875969,  0.21884013,  0.87927288,  0.27757889,  0.51019342,
        -0.18196021,  0.29735775,  0.7308629 , -0.14238595, -0.22325607,
        -0.17866826,  0.81780319, -0.73890474,  0.5

In [5]:
def wrap_data(batch_size, seed, x_train, y_train, x_valid, y_valid, x_test, y_test):
    train_set = DataProvider(inputs=x_train, targets=y_train, seed=seed)
    train_data_local = torch.utils.data.DataLoader(train_set,
                                                   batch_size=batch_size,
                                                   num_workers=2,
                                                   sampler=ImbalancedDatasetSampler(train_set))

    valid_set = DataProvider(inputs=x_valid, targets=y_valid, seed=seed)
    valid_data_local = torch.utils.data.DataLoader(valid_set,
                                                   batch_size=batch_size,
                                                   num_workers=2,
                                                   shuffle=False)

    test_set = DataProvider(inputs=x_test, targets=y_test, seed=seed)
    test_data_local = torch.utils.data.DataLoader(test_set,
                                                  batch_size=batch_size,
                                                  num_workers=2,
                                                  shuffle=False)

    return train_data_local, valid_data_local, test_data_local

def fetch_model(model, embedding_level, input_shape_local, dropout):
    if model == 'MLP':
        return multi_layer_perceptron(input_shape_local)
    if embedding_level == 'word':
        return word_cnn(input_shape_local, dropout)
    elif embedding_level == 'character':
        return character_cnn(input_shape_local)
    else:
        raise ValueError("Model key not found {}".format(embedding_level))


def fetch_model_parameters(input_shape_local):
    model_local, criterion_local, optimizer_local = fetch_model(model='CNN',
                                                                embedding_level='word',
                                                                input_shape_local=input_shape_local,
                                                                dropout=0.5)

    scheduler_local = optim.lr_scheduler.CosineAnnealingLR(optimizer_local, T_max=100, eta_min=0.0001)
    return model_local, criterion_local, optimizer_local, scheduler_local



In [6]:
train_data, valid_data, test_data = wrap_data(64, 28, **data)

In [7]:
input_shape = tuple([64] + list(np.array(data['x_train']).shape)[1:])

In [8]:
def run_train_iter(model, device, optimizer, criterion, x, y, stats, experiment_key='train'):
    """
    Receives the inputs and targets for the model and runs a training iteration. Returns loss and accuracy metrics.
    :param x: The inputs to the model. A numpy array of shape batch_size, channels, height, width
    :param y: The targets for the model. A numpy array of shape batch_size, num_classes
    :return: the loss and accuracy for this batch
    """
    # sets model to training mode
    # (in case batch normalization or other methods have different procedures for training and evaluation)
    model.train()
    if type(x) is np.ndarray:
        x, y = torch.Tensor(x).float().to(device=self.device), torch.Tensor(y).long().to(
            device=self.device)  # convert data to pytorch tensors and send to the computation device

    x = x.to(device)
    x = x.float()
    y = y.to(device)
    optimizer.zero_grad()  # set all weight grads from previous training iters to 0
    out = model.forward(x)  # forward the data in the model
    # loss = F.cross_entropy(input=out, target=y)  # compute loss
    loss = criterion(out, y)
    loss.backward()  # backpropagate to compute gradients for current iter loss

    optimizer.step()  # update network parameters
    _, predicted = torch.max(out.data, 1)  # get argmax of predictions
    accuracy = np.mean(list(predicted.eq(y.data).cpu()))  # compute accuracy
    stats['{}_acc'.format(experiment_key)].append(accuracy)
    stats['{}_loss'.format(experiment_key)].append(loss.data.detach().cpu().numpy())

def run_evaluation_iter(model, device, optimizer, criterion, x, y, stats, experiment_key='valid'):
    """
    Receives the inputs and targets for the model and runs an evaluation iterations. Returns loss and accuracy metrics.
    :param x: The inputs to the model. A numpy array of shape batch_size, channels, height, width
    :param y: The targets for the model. A numpy array of shape batch_size, num_classes
    :return: the loss and accuracy for this batch
    """
    with torch.no_grad():
        model.eval()  # sets the system to validation mode
        if type(x) is np.ndarray:
            x, y = torch.Tensor(x).float().to(device=self.device), torch.Tensor(y).long().to(
                device=self.device)  # convert data to pytorch tensors and send to the computation device

        x = x.to(device)
        x = x.float()
        y = y.to(device)
        out = model.forward(x)  # forward the data in the model
        loss = criterion(out, y)
        # loss = F.cross_entropy(out, y)  # compute loss
        _, predicted = torch.max(out.data, 1)  # get argmax of predictions
        accuracy = np.mean(list(predicted.eq(y.data).cpu()))
        stats['{}_acc'.format(experiment_key)].append(accuracy)  # compute accuracy
        stats['{}_loss'.format(experiment_key)].append(loss.data.detach().cpu().numpy())
        

In [42]:
def save_model(model, model_save_dir, model_save_name, model_idx):
    """
    Save the network parameter state and current best val epoch idx and best val accuracy.
    :param model_save_name: Name to use to save model without the epoch index
    :param model_idx: The index to save the model with.
    :param best_validation_model_idx: The index of the best validation model to be stored for future use.
    :param best_validation_model_acc: The best validation accuracy to be stored for use at test time.
    :param model_save_dir: The directory to store the state at.
    :param state: The dictionary containing the system state.

    """
    # Save state each epoch
    path = os.path.join(model_save_dir, "{}_{}".format(model_save_name, str(model_idx)))
    torch.save(model.state_dict(), f=path)
    

def load_model(model, model_save_dir, model_save_name, model_idx):
    """
    Load the network parameter state and the best val model idx and best val acc to be compared with the future val accuracies, in order to choose the best val model
    :param model_save_dir: The directory to store the state at.
    :param model_save_name: Name to use to save model without the epoch index
    :param model_idx: The index to save the model with.
    """
    path = os.path.join(model_save_dir, "{}_{}".format(model_save_name, str(model_idx)))
    checkpoint = torch.load(f=path)
    # freeze parameters
    model.load_state_dict(checkpoint)
    for parameter in model.parameters():
        parameter.requires_grad = False
    return model 

In [10]:
model, criterion, optimizer, scheduler = fetch_model_parameters(input_shape)

Building basic block of ConvolutionalNetwork using input shape torch.Size([64, 400, 17])
Block is built, output volume is torch.Size([64, 4])


In [13]:
model

CNN(
  (drop): Dropout(p=0.5)
  (layer_dict): ModuleDict(
    (conv_0): Conv1d(400, 32, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (batch_norm_0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_1): Conv1d(32, 32, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (batch_norm_1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_2): Conv1d(64, 32, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (batch_norm_2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (logit_linear_layer): Linear(in_features=96, out_features=4, bias=False)
)

In [96]:
from collections import OrderedDict, defaultdict
import tqdm

model, criterion, optimizer, scheduler = fetch_model_parameters(input_shape)
device = torch.device('cpu')
train_stats = OrderedDict()
model_params = {}
num_epochs = 5
for epoch_idx in range(num_epochs):
    epoch_start_time = time.time()
    epoch_stats = defaultdict(list)
    with tqdm.tqdm(total=len(train_data)) as pbar_train:  # create a progress bar for training
        for idx, (x, y) in enumerate(train_data):  # get data batches
            run_train_iter(model, device, optimizer, criterion, x=x, y=y, stats=epoch_stats)  # take a training iter step
            pbar_train.update(1)
            pbar_train.set_description("loss: {:.4f}, accuracy: {:.4f}".format(epoch_stats['train_loss'][-1],
                                                                               epoch_stats['train_acc'][-1]))

    with tqdm.tqdm(total=len(valid_data)) as pbar_val:  # create a progress bar for validation
        for x, y in valid_data:  # get data batches
            run_evaluation_iter(model, device, optimizer, criterion, x=x, y=y, stats=epoch_stats)  # run a validation iter
            pbar_val.update(1)  # add 1 step to the progress bar
            pbar_val.set_description("loss: {:.4f}, accuracy: {:.4f}".format(epoch_stats['valid_loss'][-1],
                                                                             epoch_stats['valid_acc'][-1]))
    #evaluate test here
    with tqdm.tqdm(total=len(test_data)) as pbar_test:  # create a progress bar for validation
        for x, y in test_data:  # get data batches
            run_evaluation_iter(model, device, optimizer, criterion, x=x, y=y, stats=epoch_stats, experiment_key="test_experiment")  # run a validation iter
            pbar_test.update(1)  # add 1 step to the progress bar
            pbar_test.set_description("loss: {:.4f}, accuracy: {:.4f}".format(epoch_stats['test_experiment_loss'][-1],
                                                                  epoch_stats['test_experiment_acc'][-1]))
    
    model_params[epoch_idx] = list(model.parameters())
    save_model(model, '', 'testing', epoch_idx)

  0%|          | 0/584 [00:00<?, ?it/s]

Building basic block of ConvolutionalNetwork using input shape torch.Size([64, 400, 17])
Block is built, output volume is torch.Size([64, 4])


loss: 0.6155, accuracy: 0.7917: 100%|██████████| 183/183 [00:02<00:00, 78.04it/s]


In [97]:
loaded_model = load_model(model, '', 'testing', 0)

In [98]:
torch.equal(list(loaded_model.parameters())[0], model_params[0][0])

True

In [34]:
torch.equal(model_first_parameters[0], final_model_parameters[0])

True

In [35]:
for parameter in model.parameters():
    parameter.requires_grad = False

In [36]:
torch.equal(model_first_parameters[0], final_model_parameters[0])

True

In [37]:
final_model_parameters = list(model.parameters())


In [38]:
torch.equal(model_first_parameters[0], final_model_parameters[0])

True